In [66]:
import pandas as pd
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing

In [114]:
df=pd.read_csv('student_data.csv').dropna()
print(df.columns)
gpa =df['gpa']
df['A']=df['gpa'].apply(lambda x: 1 if ((x<=4.0) & (x>3.7)) else 0)
df['A-']=df['gpa'].apply(lambda x: 1 if ((x<=3.7) & (x>3.3)) else 0)
df['B+']=df['gpa'].apply(lambda x: 1 if (x<=3.3) & (x>3.0) else 0)
df['B']=df['gpa'].apply(lambda x: 1 if (x<=3.0) & (x>2.7) else 0)
df['B-']=df['gpa'].apply(lambda x: 1 if (x<=2.7) & (x>2.3) else 0)
df['C+']=df['gpa'].apply(lambda x: 1 if (x<=2.3) & (x>2.0) else 0)
df['C']=df['gpa'].apply(lambda x: 1 if (x<=2.0) & (x>1.7) else 0)
df['C-']=df['gpa'].apply(lambda x: 1 if (x<=1.7) & (x>1.3) else 0)
df['D+']=df['gpa'].apply(lambda x: 1 if (x<=1.3) & (x>1.2) else 0)
df['D']=df['gpa'].apply(lambda x: 1 if (x<=1.2) & (x>1.1) else 0)
df['D-']=df['gpa'].apply(lambda x: 1 if (x<=1.1) & (x>.9) else 0)
df['F']=df['gpa'].apply(lambda x: 1 if (x<1) else 0)

df['Grade']=''

for key, value in df.iterrows():
    if value['A']==1:
        df.loc[key,'Grade']='A'
    elif value['A-']==1:
        df.loc[key,'Grade']='A-'
    elif value['B+']==1:
        df.loc[key,'Grade']='B+'
    elif value['B']==1:
        df.loc[key,'Grade']='B'
    elif value['B-']==1:
        df.loc[key,'Grade']='B-'
    elif value['C+']==1:
        df.loc[key,'Grade']='C+'
    elif value['C']==1:
        df.loc[key,'Grade']='C'
    elif value['C-']==1:
        df.loc[key,'Grade']='C-'
    elif value['D+']==1:
        df.loc[key,'Grade']='D+'
    elif value['D']==1:
        df.loc[key,'Grade']='D'
    elif value['D-']==1:
        df.loc[key,'Grade']='D-'
    elif value['F']==1:
        df.loc[key,'Grade']='F'
        
encoder=preprocessing.LabelEncoder()        
df['Grade_enc']=encoder.fit_transform(df['Grade'])

NUMERIC=['gre','Grade_enc']
y=df['admit'].astype('long')
X=df[NUMERIC]

Index(['Unnamed: 0', 'admit', 'gre', 'gpa', 'ses', 'Gender_Male', 'Race',
       'rank', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-',
       'F', 'Grade', 'Grade_enc'],
      dtype='object')


In [130]:
N_MULT_FACTOR=8

class Net2(nn.Module):    
    def __init__(self, n_feature, n_hidden, n_output, n_cnn_kernel, n_mult_factor=N_MULT_FACTOR):
        super(Net2, self).__init__()
        self.n_feature=n_feature
        self.n_hidden=n_hidden
        self.n_output= n_output 
        self.n_cnn_kernel=n_cnn_kernel
        self.n_mult_factor=n_mult_factor
        
         self.n_l2_hidden=self.n_hidden * (self.n_mult_factor - self.n_cnn_kernel + 3)
#         self.n_out_hidden=int (self.n_l2_hidden/2)
                        
        self.l1 = nn.Sequential(
            torch.nn.Linear(self.n_feature, self.n_hidden),
            torch.nn.Dropout(p=1 -.85),            
            torch.nn.LeakyReLU (0.1),            
            torch.nn.BatchNorm1d(self.n_hidden, eps=1e-05, momentum=0.1, affine=True)            
        )                
        self.c1= nn.Sequential(            
            torch.nn.Conv1d(self.n_feature, self.n_hidden, 
                            kernel_size=(self.n_cnn_kernel,), stride=(1,), padding=(1,)),
            torch.nn.Dropout(p=1 -.75),            
            torch.nn.LeakyReLU (0.1),
            torch.nn.BatchNorm1d(self.n_hidden, eps=1e-05, momentum=0.1, affine=True)        
        )                        
        self.out = nn.Sequential(
            torch.nn.Linear(self.n_l2_hidden,
                            self.n_output),  
        )                
        self.sig=nn.Sigmoid()

        
    def forward(self, x):
        varSize=x.data.shape[0] # must be calculated here in forward() since its is a dynamic size        
        x=self.l1(x)                
        # for CNN        
        x = x.view(varSize,self.n_feature,self.n_mult_factor)
        x=self.c1(x)
        # for Linear layer
        x = x.view(varSize, self.n_hidden * (self.n_mult_factor -self.n_cnn_kernel + 3))
#         x=self.l2(x)                    
        x=self.out(x)   
        x=self.sig(x)
        return x

In [131]:
def YnumpyToTensor(y_data_np):    
    y_data_np=y_data_np.reshape((y_data_np.shape[0],1)) # Must be reshaped for PyTorch!
    print(y_data_np.shape)
    print(type(y_data_np))

    Y_tensor = Variable(torch.from_numpy(y_data_np)).type(torch.FloatTensor)  # BCEloss requires Float        

    print(type(Y_tensor.data)) # should be 'torch.cuda.FloatTensor'
    print(y_data_np.shape)
    print(type(y_data_np))    
    return Y_tensor

In [149]:
N_FEATURES=X.shape[1]
N_HIDDEN=N_FEATURES * N_MULT_FACTOR
N_CNN_KERNEL=5
net = Net2(n_feature=N_FEATURES, n_hidden=N_HIDDEN, n_output=1, n_cnn_kernel=N_CNN_KERNEL) 

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(), lr=0.01)

X_tensor=torch.Tensor(np.array(X)).reshape(len(X),2)
#print(X_tensor)
Target=torch.Tensor(y).long()
print(Target)

#print(Target)
for epoch in range(1000):
    print("Epoch",epoch)
    optimizer.zero_grad()
    #inputs=inputs.view(-1,128*4*4)
    outputs=net(X_tensor)
    #print(outputs)
    #print("Target",Target,"Outputs",outputs)
    loss=criterion(outputs,Target)
    loss.backward()
    optimizer.step()
    if epoch % 1 == 0:
        print('[%d] loss: %.3f' % (epoch, loss.item()))
        #print(net.weight.data.numpy())

tensor([0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

IndexError: Target 1 is out of bounds.